<a href="https://colab.research.google.com/github/AbeHandler/AbeHandler.github.io/blob/master/sa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### The plan

1. Compute sa for each shard
2. Merge them w/ a mem map low memory merge
3. Shard the big SA. The sequences that are similar will go in the same shard
4. Search those shards in parallel

In [1]:
from pydivsufsort import divsufsort, kasai

def compute_sa(text):
    return sorted(range(len(text)), key=lambda i: text[i:])

def compute_lcp(text, sa):
    lcp = [0] * len(sa)
    for i in range(1, len(sa)):
        # find common prefix length between suffixes at sa[i] and sa[i-1]
        a, b = text[sa[i]:], text[sa[i-1]:]
        h = 0
        while h < len(a) and h < len(b) and a[h] == b[h]:
            h += 1
        lcp[i] = h
    return lcp

s = "aabaa"
s1 = f"{s}document stuff"
s2 = f"{s}lorem"
text = s1 + "$" + s2  # "aabaa document stuff$aabaa lorem"

# Compute SA and LCP
sa = divsufsort(text)
lcp = kasai(text, sa)

THRESHOLD = 5

# Display entries where LCP >= 2 and show the two common characters
print(f"{'i':>2} {'SA[i]':>5} {'SA[i-1]':>7} {'LCP[i]':>7}  {f'Common ({THRESHOLD} chars)':>20}  {'Suffix_i':>12}  {'Suffix_prev'}")
print("-" * 100)
for i in range(1, len(sa)):
    if lcp[i] >= THRESHOLD:
        pos_i = sa[i]
        pos_prev = sa[i-1]
        common = text[pos_i:pos_i + THRESHOLD]
        suffix_i = text[pos_i:]
        suffix_prev = text[pos_prev:]
        print(f"{i:2} {pos_i:5} {pos_prev:7} {lcp[i]:7}  {common!r:20}  {suffix_i!r:12}  {suffix_prev!r}")

 i SA[i] SA[i-1]  LCP[i]      Common (5 chars)      Suffix_i  Suffix_prev
----------------------------------------------------------------------------------------------------
 2     0      19       5  'aabaa'               'aabaadocument stuff$aabaalorem'  '$aabaalorem'


In [10]:
import numpy as np
import mmap

sa1 = divsufsort(s1).astype(np.uint64)
sa1.tofile("sa1.bin")

sa2 = divsufsort(s2).astype(np.uint64)
sa2.tofile("sa2.bin")


sa1_mm = np.memmap("sa1.bin", dtype=np.uint64, mode="r")
sa2_mm = np.memmap("sa2.bin", dtype=np.uint64, mode="r")

# Write the concatenated text to disk for mmap
with open("text.bin", "wb") as f:
    f.write((s1 + s2).encode())

# Memory-map the text
f_text = open("text.bin", "rb")
text_mm = mmap.mmap(f_text.fileno(), 0, access=mmap.ACCESS_READ)

# 4) Merge the two suffix arrays without loading fully into RAM
len1 = len(s1)
n1, n2 = sa1_mm.shape[0], sa2_mm.shape[0]
merged_len = n1 + n2

merged_sa = np.memmap("merged_sa.bin", dtype=np.uint64, mode="w+", shape=(merged_len,))

i = j = k = 0
while i < n1 and j < n2:
    idx1 = int(sa1_mm[i])
    idx2 = int(sa2_mm[j]) + len1
    # compare suffixes via mmap
    if text_mm[idx1:] < text_mm[idx2:]:
        merged_sa[k] = idx1
        i += 1
    else:
        merged_sa[k] = idx2
        j += 1
    k += 1

# drain remainders
while i < n1:
    merged_sa[k] = int(sa1_mm[i])
    i += 1; k += 1
while j < n2:
    merged_sa[k] = int(sa2_mm[j]) + len1
    j += 1; k += 1

merged_sa.flush()
print("Merged suffix array written to 'merged_sa.bin'")

merged_sa = np.fromfile("merged_sa.bin", dtype=np.uint64)
merged_sa_correct = divsufsort(s1 + s2).astype(np.uint64)

Merged suffix array written to 'merged_sa.bin'


In [13]:
merged_sa

array([13,  0, 19,  3, 22,  1, 20,  4, 23,  2, 21,  7,  5, 27, 10, 18, 17,
       24, 28,  9, 11,  6, 25, 26, 14, 12, 15, 16,  8], dtype=uint64)